In [16]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@File    :   main.py
@Time    :   2021/03/30 11:40:05
@Author  :   bin.wang
@Version :   1.0
'''

# here put the import lib
import os
import time

import pickle
from tqdm import tqdm

import numpy as np

from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import mahalanobis

from skimage import morphology
from skimage.segmentation import mark_boundaries

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt
import matplotlib


import torch
import torch.nn.functional as F

from torch.utils.data import DataLoader

# my imports
import my_parser
import cw_saab as sb
import mvtec_data_loader as mvtec_loader

In [29]:
# Kaitai: SVDD imports
import sys
from svdd import SVDD
from visualize import Visualization as draw

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing
import scipy.io as sio

parameters = {"positive penalty": 0.9,
                "negative penalty": [],
                "kernel": {"type": 'gauss', "width": 1/80},
                "option": {"display": 'on'}}

# # construct an SVDD model
# svdd = SVDD(parameters)

# # train SVDD model
# svdd.train(trainData, trainLabel)

# # test SVDD model
# distance, accuracy = svdd.test(testData, testLabel)

In [79]:
KERNEL = [5,5,5,5,5]
KEEP_COMPONENTS = [5,5,5,5,5]
MAX_GMM_SAMPLES = 100000
GMM_COMPONENTS = 5
#DISTANCE_MEASURE = ['PIXEL_GAUSS', 'KMEANS', 'GLOBAL_GAUSS'] # 'PIXEL_GAUSS', 'KMEANS', 'GLOBAL_GAUSS'
DISTANCE_MEASURE = 'PIXEL_GAUSS' # 'PIXEL_GAUSS', 'KMEANS', 'GLOBAL_GAUSS'
DISTANCE_MEASURE = 'SVDD'

In [126]:
import argparse

parser = argparse.ArgumentParser('AnomalyHop')
parser.add_argument('--data_path', type=str, default='/home/mcl/Desktop/AnomalyHop/datasets/mvtec_anomaly_detection')
parser.add_argument('--save_path', type=str, default='/home/mcl/Desktop/AnomalyHop/datasets/mvtec_result')

args = parser.parse_args(args=[])
print("\n######   Arguments:   ######\n", args)

total_roc_auc = []
total_pixel_roc_auc = []

all_results = {}

# data loader
class_name = 'capsule'
train_dataset = mvtec_loader.MVTecDataset(args.data_path, class_name=class_name, is_train=True)
train_dataloader = DataLoader(train_dataset, batch_size=32, pin_memory=True)
test_dataset = mvtec_loader.MVTecDataset(args.data_path, class_name=class_name, is_train=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, pin_memory=True)



######   Arguments:   ######
 Namespace(data_path='/home/mcl/Desktop/AnomalyHop/datasets/mvtec_anomaly_detection', save_path='/home/mcl/Desktop/AnomalyHop/datasets/mvtec_result')


In [127]:
# - - - - - - - - - - - - - - - - - - - - Training - - - - - - - - - - - - - - - - - - - - - - - - 

# extract train set features
train_feature_filepath = os.path.join(args.save_path, 'train_%s.pkl' % class_name)


print("\n######   Prepare Training Data:   ######\n")
all_train_input = []

for (x, _, _) in tqdm(train_dataloader, '| feature extraction | train | %s |' % class_name):
    x = x.numpy()
    all_train_input.append(x)

all_train_input = np.concatenate(all_train_input)

print("\n######   Saak Training:   ######\n")

# all_train_input = np.random.rand(209, 3, 224, 224) # okay
# all_train_input # (209, 3, 224, 224) # no okay


sb_params, sb_feature_all, sb_feature_last = sb.multi_saab_chl_wise(all_train_input,
                                                                    [1,1,1,1,1], # stride
                                                                    KERNEL, # kernel
                                                                    [1,1,1,1,1], # dilation
                                                                    KEEP_COMPONENTS,
                                                                    0.125,
                                                                    padFlag = [False,False,False,False,False],
                                                                    recFlag = True,
                                                                    collectFlag = True)
# show all hops dimensions
for i in range(len(sb_feature_all)):
    print('stage ', i, ': ', sb_feature_all[i].shape)

| feature extraction | train | capsule |:   0%|          | 0/7 [00:00<?, ?it/s]
######   Prepare Training Data:   ######

| feature extraction | train | capsule |: 100%|██████████| 7/7 [00:10<00:00,  1.57s/it]

######   Saak Training:   ######

--------stage 0 --------
Sample/cuboid_previous.shape: (219, 3, 224, 224)
Sample/cuboid_forNextStage.shape: (219, 3, 224, 224)
Sample/cuboid_toNextStage.shape: (219, 18, 220, 220)
--------stage 1 --------
Sample/cuboid_previous.shape: (219, 18, 220, 220)
Sample/cuboid_forNextStage.shape: (219, 9, 220, 220)
Sample/max_pooling.shape: (219, 9, 110, 110)
Sample/cuboid_toNextStage.shape: (219, 54, 106, 106)
--------stage 2 --------
Sample/cuboid_previous.shape: (219, 54, 106, 106)
Sample/cuboid_forNextStage.shape: (219, 20, 106, 106)
Sample/max_pooling.shape: (219, 20, 53, 53)
Sample/cuboid_toNextStage.shape: (219, 120, 49, 49)
--------stage 3 --------
Sample/cuboid_previous.shape: (219, 120, 49, 49)
Sample/cuboid_forNextStage.shape: (219, 34, 49, 49

In [128]:
train_outputs = []

if 'PIXEL_GAUSS' == DISTANCE_MEASURE:
    for i_layer in range(len(sb_feature_all)):

        train_layer_i_feature = sb_feature_all[i_layer]
        train_layer_i_feature = np.array(train_layer_i_feature)
        B, C, H, W = train_layer_i_feature.shape

        train_layer_i_feature = train_layer_i_feature.reshape(B, C, H * W)
    
        # gaussian distance measure            
        mean = np.mean(train_layer_i_feature, 0)
        cov = np.zeros((C, C, H * W))
        I = np.identity(C)
        for i in range(H * W):
            cov[:, :, i] = np.cov(train_layer_i_feature[:, :, i], rowvar=False) + 0.01 * I
        
        train_outputs.append([mean, cov])

if 'SVDD' == DISTANCE_MEASURE:
    for i_layer in [2]:

        train_layer_i_feature = sb_feature_all[i_layer]
        train_layer_i_feature = np.array(train_layer_i_feature)
        B, C, H, W = train_layer_i_feature.shape

        train_layer_i_feature = train_layer_i_feature.reshape(B, C, H * W)  

        # pixel SVDD
        trainLabel =np.ones((train_layer_i_feature.shape[0], 1))

        train_output_layer_i = []           
        for i in range(H * W):
            # construct an SVDD model
            svdd = SVDD(parameters)

            # train SVDD model
            svdd.train(train_layer_i_feature[:, :, i], trainLabel)
            train_output_layer_i.append(svdd)
        
        train_outputs.append(train_output_layer_i)


 12
time cost        = 0.0218 s
obj              = -0.8988
pData            = 100.0000 %
nData            = 0.0000 %
nSVs             = 89
radio of nSVs    = 40.6393 %
accuracy         = 94.5205 %




*** SVDD model training finished ***

iter             = 12
time cost        = 0.0227 s
obj              = -0.8985
pData            = 100.0000 %
nData            = 0.0000 %
nSVs             = 47
radio of nSVs    = 21.4612 %
accuracy         = 96.8037 %




*** SVDD model training finished ***

iter             = 12
time cost        = 0.0221 s
obj              = -0.8985
pData            = 100.0000 %
nData            = 0.0000 %
nSVs             = 112
radio of nSVs    = 51.1416 %
accuracy         = 96.8037 %




*** SVDD model training finished ***

iter             = 12
time cost        = 0.0220 s
obj              = -0.8981
pData            = 100.0000 %
nData            = 0.0000 %
nSVs             = 60
radio of nSVs    = 27.3973 %
accuracy         = 96.8037 %




*** SVDD model training fin

In [129]:

# - - - - - - - - - - - - - - - - - - - - Testing - - - - - - - - - - - - - - - - - - - - - - - - 
gt_list = []
gt_mask_list = []
test_imgs = []

for (x, y, mask) in tqdm(test_dataloader, '| feature extraction | test | %s |' % class_name):
    
    test_imgs.extend(x.cpu().detach().numpy())
    gt_list.extend(y.cpu().detach().numpy())
    gt_mask_list.extend(mask.cpu().detach().numpy())

test_imgs = np.stack(test_imgs)

_, sb_test_feature_all, _ = sb.inference_chl_wise(sb_params,
                                                    test_imgs, 
                                                    True, 
                                                    -1, 
                                                    len(KERNEL)-1,
                                                    collectFlag=True)

# show all hops dimensions
for i in range(len(sb_test_feature_all)):
    print('stage ', i, ': ', sb_test_feature_all[i].shape)

| feature extraction | test | capsule |: 100%|██████████| 5/5 [00:06<00:00,  1.35s/it]
--------stage 0 --------
Sample/cuboid_previous.shape: (132, 3, 224, 224)
Sample/cuboid_toNextStage.shape: (132, 18, 220, 220)
--------stage 1 --------
Sample/cuboid_previous.shape: (132, 18, 220, 220)
Sample/max_pooling.shape: (132, 9, 110, 110)
Sample/cuboid_toNextStage.shape: (132, 54, 106, 106)
--------stage 2 --------
Sample/cuboid_previous.shape: (132, 54, 106, 106)
Sample/max_pooling.shape: (132, 20, 53, 53)
Sample/cuboid_toNextStage.shape: (132, 120, 49, 49)
--------stage 3 --------
Sample/cuboid_previous.shape: (132, 120, 49, 49)
Sample/max_pooling.shape: (132, 34, 25, 25)
Sample/cuboid_toNextStage.shape: (132, 204, 21, 21)
--------stage 4 --------
Sample/cuboid_previous.shape: (132, 204, 21, 21)
Sample/max_pooling.shape: (132, 69, 11, 11)
Sample/cuboid_toNextStage.shape: (132, 414, 7, 7)
stage  0 :  (132, 18, 220, 220)
stage  1 :  (132, 54, 106, 106)
stage  2 :  (132, 120, 49, 49)
stage  3 

In [130]:
scores = []

if 'PIXEL_GAUSS' == DISTANCE_MEASURE:
    for i_layer in range(len(sb_test_feature_all)):
        test_layer_i_feature = sb_test_feature_all[i_layer]
        test_layer_i_feature = np.array(test_layer_i_feature)

        B, C, H, W = test_layer_i_feature.shape
        test_layer_i_feature = test_layer_i_feature.reshape(B, C, H * W)

        # gaussian distance measure           
        dist_list = []
        for i in range(H * W):
            mean = train_outputs[i_layer][0][:, i]
            conv_inv = np.linalg.inv(train_outputs[i_layer][1][:, :, i])
            dist = [mahalanobis(sample[:, i], mean, conv_inv) for sample in test_layer_i_feature]
            dist_list.append(dist)

        dist_list = np.array(dist_list).transpose(1, 0).reshape(B, H, W)
    
        # upsample
        dist_list = torch.tensor(dist_list)
        score_map = F.interpolate(dist_list.unsqueeze(1), size=x.size(2), mode='bilinear',
                                align_corners=False).squeeze().numpy()

        # apply gaussian smoothing on the score map
        for i in range(score_map.shape[0]):
            score_map[i] = gaussian_filter(score_map[i], sigma=4)
        # Normalization
        max_score = score_map.max()
        min_score = score_map.min()
        score = (score_map - min_score) / (max_score - min_score)
        scores.append(score) # all scores from different hop features

if 'SVDD' == DISTANCE_MEASURE:
    for i_layer in [2]:
        test_layer_i_feature = sb_test_feature_all[i_layer]
        test_layer_i_feature = np.array(test_layer_i_feature)

        B, C, H, W = test_layer_i_feature.shape
        test_layer_i_feature = test_layer_i_feature.reshape(B, C, H * W)

        # svdd distance measure 
        train_output_layer_i = train_outputs[i_layer-2]
        testLabel =np.ones((test_layer_i_feature.shape[0], 1))

        dist_list = []
        for i in range(H * W):
            # mean = train_outputs[i_layer][0][:, i]
            # conv_inv = np.linalg.inv(train_outputs[i_layer][1][:, :, i])
            # dist = [mahalanobis(sample[:, i], mean, conv_inv) for sample in test_layer_i_feature]

            # test SVDD model
            svdd = train_output_layer_i[i]
            dist, _ = svdd.test(test_layer_i_feature[:,:,i], testLabel)
            dist_list.append(dist)

        dist_list = np.array(dist_list).transpose(1, 0).reshape(B, H, W)
    
        # upsample
        dist_list = torch.tensor(dist_list)
        score_map = F.interpolate(dist_list.unsqueeze(1), size=x.size(2), mode='bilinear',
                                align_corners=False).squeeze().numpy()

        # apply gaussian smoothing on the score map
        for i in range(score_map.shape[0]):
            score_map[i] = gaussian_filter(score_map[i], sigma=4)
        # Normalization
        max_score = score_map.max()
        min_score = score_map.min()
        score = (score_map - min_score) / (max_score - min_score)
        scores.append(score) # all scores from different hop features



        = 0.0042 s
accuracy         = 60.6061 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 62.8788 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 64.3939 %




*** SVDD model test finished ***

time cost        = 0.0043 s
accuracy         = 60.6061 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 67.4242 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 68.9394 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 68.1818 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 68.1818 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 66.6667 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 62.1212 %




*** SVDD model test finished ***

time cost        = 0.0042 s
accuracy         = 57.5758 %




In [131]:
dist_list

tensor([[[0.9544, 0.9550, 0.9546,  ..., 0.9464, 0.9443, 0.9440],
         [0.9546, 0.9547, 0.9549,  ..., 0.9469, 0.9445, 0.9445],
         [0.9551, 0.9551, 0.9549,  ..., 0.9478, 0.9462, 0.9464],
         ...,
         [0.9525, 0.9521, 0.9534,  ..., 0.9376, 0.9385, 0.9381],
         [0.9544, 0.9540, 0.9548,  ..., 0.9384, 0.9392, 0.9383],
         [0.9556, 0.9551, 0.9555,  ..., 0.9398, 0.9402, 0.9393]],

        [[0.9549, 0.9537, 0.9533,  ..., 0.9522, 0.9532, 0.9521],
         [0.9548, 0.9535, 0.9531,  ..., 0.9528, 0.9540, 0.9526],
         [0.9548, 0.9528, 0.9528,  ..., 0.9529, 0.9533, 0.9529],
         ...,
         [0.9517, 0.9503, 0.9482,  ..., 0.9412, 0.9417, 0.9414],
         [0.9520, 0.9510, 0.9491,  ..., 0.9414, 0.9412, 0.9407],
         [0.9530, 0.9515, 0.9503,  ..., 0.9414, 0.9411, 0.9409]],

        [[0.9597, 0.9596, 0.9613,  ..., 0.9554, 0.9569, 0.9576],
         [0.9595, 0.9602, 0.9612,  ..., 0.9553, 0.9579, 0.9596],
         [0.9591, 0.9598, 0.9614,  ..., 0.9572, 0.9593, 0.

In [132]:

# compute final score for all images
all_scores = []
all_scores.extend(scores)

#import pdb; pdb.set_trace()

#all_scores.extend()
#scores_final = np.mean(np.stack(scores), 0) # average all hop heat map 
#scores_final = np.mean(np.stack(k_means_scores), 0) # average all hop heat map
scores_final = np.mean(np.stack(all_scores), 0) # average all hop heat map


# calculate image-level ROC AUC score
img_scores = scores_final.reshape(scores_final.shape[0], -1).max(axis=1)
gt_list = np.asarray(gt_list)
#fpr, tpr, _ = roc_curve(gt_list, img_scores)
img_roc_auc = roc_auc_score(gt_list, img_scores)
total_roc_auc.append(img_roc_auc)
print('image ROCAUC: %.3f' % (img_roc_auc))
#fig_img_rocauc.plot(fpr, tpr, label='%s img_ROCAUC: %.3f' % (class_name, img_roc_auc))

# get optimal threshold
gt_mask = np.asarray(gt_mask_list)
precision, recall, thresholds = precision_recall_curve(gt_mask.flatten(), scores_final.flatten())
a = 2 * precision * recall
b = precision + recall
f1 = np.divide(a, b, out=np.zeros_like(a), where=b != 0)
threshold = thresholds[np.argmax(f1)]

# calculate per-pixel level ROCAUC
#fpr, tpr, _ = roc_curve(gt_mask.flatten(), scores.flatten())
per_pixel_rocauc = roc_auc_score(gt_mask.flatten(), scores_final.flatten())
total_pixel_roc_auc.append(per_pixel_rocauc)
print('pixel ROCAUC: %.3f' % (per_pixel_rocauc))

# #fig_pixel_rocauc.plot(fpr, tpr, label='%s ROCAUC: %.3f' % (class_name, per_pixel_rocauc))
# save_dir = args.save_path + '/' + f'pictures_' + class_name
# os.makedirs(save_dir, exist_ok=True)
# plot_fig(test_imgs, scores_final, gt_mask_list, threshold, save_dir, class_name)

# all_results[class_name] = {'image ROCAUC: ': img_roc_auc, 'pixel ROCAUC: ': per_pixel_rocauc}

image ROCAUC: 0.761
pixel ROCAUC: 0.939


In [94]:
# bottle only use 3rd stage(126d)
# image ROCAUC: 0.962
# pixel ROCAUC: 0.949
# capsule only use 3rd stage(126d)
# image ROCAUC: 0.761
# pixel ROCAUC: 0.939